In [1]:
import os
from urllib import request
from bs4 import BeautifulSoup 
import time
import gzip
import numpy as np
import pandas
from pprint import pprint
#import io
data_dir = '/home/idies/workspace/Temporary/raddick/cra_scratch/'
jobs_dir = data_dir + 'lodes_wac/'
baltimore_dir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/baltimore/'

this_state = 'wy'
this_state_i = 62


g = 0
print('Done!')
os.getcwd()

Done!


'/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act'

In [2]:
s = time.time()
y = os.listdir(data_dir)
hasdir = [x for x in y if x=='lodes_wac']
if (len(hasdir) == 0):
    os.mkdir(jobs_dir)
e = time.time()
g = g + (e-s)
print('Done in {0:,.3f} seconds!'.format(e-s))
#os.listdir(jobs_dir)
os.listdir(jobs_dir)

Done in 0.002 seconds!


['jobs_data_ak.csv',
 'jobs_data_al.csv',
 'jobs_data_ar.csv',
 'jobs_data_dc.csv',
 'jobs_data_az.csv',
 'jobs_data_ca_0.csv',
 'jobs_data_ca_1.csv',
 'jobs_data_ca_2.csv',
 'jobs_data_ca_3.csv',
 'jobs_data_co.csv',
 'jobs_data_ct.csv',
 'jobs_data_de.csv',
 'jobs_data_fl_0.csv',
 'jobs_data_fl_1.csv',
 'jobs_data_fl_2.csv',
 'jobs_data_fl_3.csv',
 'jobs_data_ga_0.csv',
 'jobs_data_ga_1.csv',
 'jobs_data_ga_2.csv',
 'jobs_data_ga_3.csv',
 'jobs_data_hi.csv',
 'jobs_data_ia.csv',
 'jobs_data_id.csv',
 'jobs_data_il_0.csv',
 'jobs_data_il_1.csv',
 'jobs_data_il_2.csv',
 'jobs_data_il_3.csv',
 'jobs_data_in.csv',
 'jobs_data_ks.csv',
 'jobs_data_ky.csv',
 'jobs_data_la.csv',
 'jobs_data_ma.csv',
 'jobs_data_md.csv',
 'jobs_data_me.csv',
 'jobs_data_mi.csv',
 'jobs_data_mn.csv',
 'jobs_data_mo.csv',
 'jobs_data_ms.csv',
 'jobs_data_mt.csv',
 'jobs_data_nc.csv',
 'jobs_data_nd.csv',
 'jobs_data_ne.csv',
 'jobs_data_nh.csv',
 'jobs_data_nj.csv',
 'jobs_data_nm.csv',
 'jobs_data_nv.csv',
 '

# Get WAC data from census.gov and combine into one file per state

In [3]:
s = time.time()


theurl = 'https://lehd.ces.census.gov/data/lodes/LODES7/'
print('scraping data...')
page = request.urlopen(theurl)
soup = BeautifulSoup(page, 'html.parser')
page.close()

bigtable = soup.find('table')
allstaterows = bigtable.findAll('tr')

#for i in range(9,10): # Alaska: wrote 1,975,372 rows to jobs_data_ak.csv in 24 minutes 40 seconds!
#for i in range(10,11): # Alabama:
#for i in range(11,12): # Arkansas: wrote 11,346,396 rows to jobs_data_ar.csv in 72 minutes 26 seconds!
#for i in range(12,13): # Arizona: wrote 14,821,818 rows to jobs_data_az.csv in 90 minutes 31 seconds!

#for i in range(13,14): # California 0, California 1
# California 2: wrote 24,754,939 rows to jobs_data_ca_2.csv in 46 minutes 47 seconds!
# California 3: Wrote 20,696,087 rows to jobs_data_ca_3.csv in 44 minutes 35 seconds!
# Colorado: 
# Connecticut: wrote 11,443,174 rows to jobs_data_ct.csv in 79 minutes 1 seconds!
# DC: wrote 824,450 rows to jobs_data_dc.csv in 5 minutes 21 seconds!
# Delaware: wrote 2,963,580 rows to jobs_data_de.csv in 23 minutes 15 seconds!

for i in range(this_state_i, this_state_i + 1):
    allstatecells = allstaterows[i].findAll('td')
    this_state_url = theurl + allstatecells[1].find('a').get('href')+'wac/'
    print('Scraping from: {0:}'.format(this_state_url))
    #print(this_state_url)
    statepage = request.urlopen(this_state_url)
    statesoup = BeautifulSoup(statepage, 'html.parser')
    statepage.close()
    statetable = statesoup.find('table')
    filerows = statetable.findAll('tr')
    print('Parsing state {0:,.0f} of {1:,.0f}... Getting {2:,.0f} files...'.format(i, 60, len(filerows)-4))
    for j in range(3, len(filerows)-1):
        if ((np.mod(j, 100) == 0) | (j == len(filerows)-2)):
            print('\tReading file {0:,.0f} of {1:,.0f}...'.format(j, len(filerows)-4))
        filecells = filerows[j].findAll('td')
        this_file_url = this_state_url + filecells[1].find('a').get('href')
        this_file_name = jobs_dir + filecells[1].text
        with open(this_file_name, 'wb') as f:
            r = request.urlopen(this_file_url)
            f.write(r.read())
            r.close()
nFiles = len([x for x in os.listdir(jobs_dir) if '.gz' in x])
e = time.time()
g = g + (e-s)
print('Scrapered data to create {0:,.0f} files in {1:,.0f} minutes {2:,.0f} seconds!'.format(nFiles, np.floor((e-s)/60), (e-s)%60))


scraping data...
Scraping from: https://lehd.ces.census.gov/data/lodes/LODES7/wy/wac/
Parsing state 62 of 60... Getting 860 files...
	Reading file 100 of 860...
	Reading file 200 of 860...
	Reading file 300 of 860...
	Reading file 400 of 860...
	Reading file 500 of 860...
	Reading file 600 of 860...
	Reading file 700 of 860...
	Reading file 800 of 860...
	Reading file 862 of 860...
Scrapered data to create 860 files in 4 minutes 3 seconds!


### Unzip files

In [4]:
s = time.time()

print('Unzipping files...')
gzipfiles = [jobs_dir+x for x in os.listdir(jobs_dir) if ('{0:}_wac'.format(this_state) in x) and (x[-3:] == '.gz')]
print('Found {0:,.0f} gzip files...'.format(len(gzipfiles)))

for i in range(0, len(gzipfiles)):
    if ((np.mod(i, 100) == 0) or (i == len(gzipfiles) - 1)):
        print('\tUn-gzipping file {0:.0f} of {1:.0f}...'.format(i, len(gzipfiles)-1))
    csvfilename = gzipfiles[i][:-3]
    with gzip.open(gzipfiles[i], 'rb') as f:
        file_content = f.read()
    with open(csvfilename, 'wb') as f:
        f.write(file_content)
e = time.time()
g = g + (e-s)
print('Files unzipped in {0:,.0f} minutes {1:,.0f} seconds...'.format(np.floor((e-s)/60), (e-s) % 60))

s = time.time()
print('Deleting zipfiles...')
#os.listdir(jobs_dir)
gzfiles = [jobs_dir+x for x in os.listdir(jobs_dir) if x[-3:] == '.gz']
for y in gzfiles:
    os.remove(y)
e = time.time()
g = g + (e-s)

print('Deleted all .gz files in {0:,.0f} seconds...'.format(e-s))
#print('Done in {0:,.0f} seconds!'.format(g))
print('Done!')

Unzipping files...
Found 860 gzip files...
	Un-gzipping file 0 of 859...
	Un-gzipping file 100 of 859...
	Un-gzipping file 200 of 859...
	Un-gzipping file 300 of 859...
	Un-gzipping file 400 of 859...
	Un-gzipping file 500 of 859...
	Un-gzipping file 600 of 859...
	Un-gzipping file 700 of 859...
	Un-gzipping file 800 of 859...
	Un-gzipping file 859 of 859...
Files unzipped in 0 minutes 36 seconds...
Deleting zipfiles...
Deleted all .gz files in 4 seconds...
Done!


In [5]:
s = time.time()

print('Combining multiple files into a single CSV for {0:}...'.format(this_state))
jobs_df = pandas.DataFrame()
infiles = [jobs_dir+x for x in os.listdir(jobs_dir) if ('{0:}_wac'.format(this_state) in x) and (x[-4:] == '.csv')]
outfiles = [jobs_dir+x for x in os.listdir(jobs_dir) if ('{0:}'.format(this_state) in x) and ('jobs_data_' in x)]

noutputfiles = len(outfiles)
#print(noutputfiles)

if (this_state in ['ca', 'fl', 'ga', 'il', 'ny', 'tx', 'va']):   # ['ca','fl', 'il', 'ny', 'tx', 'va']
    if (noutputfiles == 0):
        startfile = 0
        last_to_read = int(np.floor(len(infiles)/4))
        indexer = 0
    elif (noutputfiles == 1):
        startfile = int(np.floor(len(infiles)/4))
        last_to_read = int(np.floor(len(infiles)/2))
        indexer = 1
    elif (noutputfiles == 2):
        startfile = int(np.floor(len(infiles)/2))
        last_to_read = int(3*np.floor(len(infiles)/4))
        indexer = 2
    elif (noutputfiles == 3):
        startfile = int(3*np.floor(len(infiles)/4))
        last_to_read =len(infiles)
        indexer = 3
else:
    startfile = 0
    last_to_read = len(infiles)

for i in range(startfile, last_to_read):
    if ((np.mod(i, 25) == 0) or (i == len(infiles) - 1)):
        print('\tCombining file {0:,.0f} of {1:,.0f}...'.format(i, last_to_read-1))
    thisyear = int(infiles[i][-8:-4])
    #print(thisyear)
    xdf = pandas.read_csv(infiles[i], low_memory=False)
    xdf = xdf.assign(year = thisyear)     
    jobs_df = jobs_df.append(xdf)
jobs_df = jobs_df.reset_index(drop=True)
jobs_df.index.name = 'rownumber'

print('writing outfile...')

if (this_state in ['ca', 'fl', 'ga', 'il', 'ny', 'tx', 'va']): 
    jobs_df.to_csv(jobs_dir+'jobs_data_{0:}_{1:.0f}.csv'.format(this_state, indexer), encoding='utf-8')
else:
    jobs_df.to_csv(jobs_dir+'jobs_data_{0:}.csv'.format(this_state), encoding='utf-8')
e = time.time()
g = g + (e-s)

if (this_state in ['ca', 'fl', 'ga', 'il', 'ny', 'tx', 'va']):  
    print('Wrote {0:,.0f} rows to jobs_data_{1:}_{2:.0f}.csv in {3:,.0f} minutes {4:,.0f} seconds!\n'.format(len(jobs_df), this_state, indexer, np.floor((e-s)/60), (e-s) % 60))
else:
    print('Done: wrote {0:,.0f} rows to jobs_data_{1:}.csv in {2:,.0f} minutes {3:,.0f} seconds!\n'.format(len(jobs_df), this_state, np.floor((g)/60), (g) % 60))

# FL 2: Wrote 15,389,748 rows to jobs_data_fl_1.csv in 24 minutes 11 seconds!
# FL 3: Wrote 11,163,047 rows to jobs_data_fl_3.csv in 21 minutes 29 seconds!

# GA 0: Wrote 9,466,819 rows to jobs_data_ga_0.csv in 14 minutes 9 seconds!
# GA 1: Wrote 7,337,549 rows to jobs_data_ga_1.csv in 10 minutes 28 seconds!
# GA 2: Wrote 7,115,544 rows to jobs_data_ga_2.csv in 10 minutes 59 seconds!
# GA 3: Wrote 5,311,130 rows to jobs_data_ga_3.csv in 9 minutes 52 seconds!
# ID: wrote 8,393,228 rows to jobs_data_id.csv in 61 minutes 56 seconds!
# IL 0: Wrote 14,595,212 rows to jobs_data_il_0.csv in 21 minutes 52 seconds!


Combining multiple files into a single CSV for wy...
	Combining file 0 of 859...
	Combining file 25 of 859...
	Combining file 50 of 859...
	Combining file 75 of 859...
	Combining file 100 of 859...
	Combining file 125 of 859...
	Combining file 150 of 859...
	Combining file 175 of 859...
	Combining file 200 of 859...
	Combining file 225 of 859...
	Combining file 250 of 859...
	Combining file 275 of 859...
	Combining file 300 of 859...
	Combining file 325 of 859...
	Combining file 350 of 859...
	Combining file 375 of 859...
	Combining file 400 of 859...
	Combining file 425 of 859...
	Combining file 450 of 859...
	Combining file 475 of 859...
	Combining file 500 of 859...
	Combining file 525 of 859...
	Combining file 550 of 859...
	Combining file 575 of 859...
	Combining file 600 of 859...
	Combining file 625 of 859...
	Combining file 650 of 859...
	Combining file 675 of 859...
	Combining file 700 of 859...
	Combining file 725 of 859...
	Combining file 750 of 859...
	Combining file 775 of

In [6]:
print('Deleting component files that we combined...')
s = time.time()
jobs_files_list = sorted([x[-6:-4] for x in os.listdir(jobs_dir) if 'jobs_' in x])
print('Found {0:.0f} state jobs files!'.format(len(jobs_files_list)))
#pprint(jobs_files_list)
files_to_delete = sorted([jobs_dir+y for y in os.listdir(jobs_dir) if ('.csv' in y) and ('jobs_' not in y)])
print('Will delete {0:,.0f} files...'.format(len(files_to_delete)))
print('Deleting...')
for z in files_to_delete:
    os.remove(z)
e = time.time()
g = g + (e-s)
print('Deleted files in {0:,.0f} minutes {1:,.0f} seconds!\n'.format(np.floor((e-s)/60), (e-s) % 60))



Deleting component files that we combined...
Found 72 state jobs files!
Will delete 860 files...
Deleting...
Deleted files in 0 minutes 4 seconds!



In [7]:
# this_state = 'il'
# i = 0
df = pandas.read_csv(jobs_dir+'jobs_data_{0:}.csv'.format(this_state))#, low_memory=False, index_col='rownumber', keep_default_na=False)
#df = pandas.read_csv(jobs_dir+'jobs_data_{0:}_{1:.0f}.csv'.format(this_state, i))#, low_memory=False, index_col='rownumber', keep_default_na=False)

df = df.set_index('rownumber')
#

print(df.groupby('year').size().sort_index())

df.sample(1).T


year
2002    210104
2003    218606
2004    227410
2005    234536
2006    239228
2007    244538
2008    247924
2009    252242
2010    248086
2011    200920
2012    224184
2013    226058
2014    225258
2015    227526
2016    226116
2017    222886
2018    224058
dtype: int64


rownumber,1303379
w_geocode,560079676002823
C000,24
CA01,0
CA02,24
CA03,0
CE01,12
CE02,6
CE03,6
CNS01,0
CNS02,0


# Get Baltimore data and write it out

s = time.time()
df = pandas.read_csv(jobs_dir+'jobs_data_{0:}.csv'.format(this_state), encoding='utf-8', low_memory=False, index_col='rownumber')
#df = pandas.read_csv(jobs_dir+'jobs_data_{0:}.csv'.format(this_state), encoding='utf-8', low_memory=False, index_col='rownumber')
#baltimore_df = df[df['w_geocode'].apply(lambda x: str(x)[0:5] == '24510')]
#baltimore_df.to_csv(baltimore_dir+'wac_jobs_df.csv', encoding='utf-8')
e = time.time()
g = g + (e-s)
print('Read {0:,.0f} rows in {1:,.0f} seconds!'.format(len(df), e-s))
df.head(2)

df.tail(3)

s = time.time()
jobs_meta_df = pandas.read_csv(data_dir+'lodes_wac_codebook.csv', encoding='utf-8', low_memory=False, index_col='varnum')
jobs_meta_df.to_csv(baltimore_dir+'wac_jobs_metadata.csv', encoding='utf-8')
e = time.time()
g = g + (e-s)
print('Wrote metadata for {0:,.0f} variables in {1:,.0f} seconds!'.format(len(jobs_meta_df), e-s))